In [17]:
import cv2 as cv
import numpy as np

In [26]:
import numpy as np
from utils.bbox_utils import *

def create_bounding_box_mask(boxes: np.ndarray) -> np.ndarray:
    """
    Create a binary mask for the given bounding boxes.

    Args:
        boxes (np.ndarray): Array of bounding boxes with shape (N, 4), where
                             each row is in the format [x1, y1, x2, y2].

    Returns:
        np.ndarray: Binary mask with shape (N, height, width).
    """

    width = np.max(boxes[..., 2])
    height = np.max(boxes[..., 3])

    boxes = BoxConverter.to_xyxy(boxes, src_format=BoxFormat.XYWH)

    N = boxes.shape[0]

    # Create a grid of coordinates
    x, y = np.meshgrid(np.arange(width), np.arange(height))

    # Reshape boxes to (N, 1, 1, 4) and replicate it over height and width
    boxes = boxes.reshape(N, 1, 1, 4)

    # Create a binary mask for each bounding box
    mask = (x >= boxes[..., 0]) & (x <= boxes[..., 2]) & \
           (y >= boxes[..., 1]) & (y <= boxes[..., 3])

    return mask

# Example usage
boxes = np.array([[10, 20, 30, 40], [50, 60, 70, 80]])
mask = create_bounding_box_mask(boxes)
print(mask.shape)  # (2, 100, 200)

(2, 80, 70)


In [29]:
print(mask[0, :].sum())

1271


In [18]:
from evaluation.error_calculator import ErrorCalculator

eval = ErrorCalculator(np.random.randint(0, 255, (100, 100, 3), dtype=np.uint8), 10)
eval.calculate_precise_single(np.zeros((30, 30, 3), dtype=np.uint8), worm_bbox=(15, 15, 30, 30), micro_bbox=(160, 000, 377, 211))

1.0

In [19]:
m = np.zeros((30, 30, 3), dtype=np.uint8)
n = np.ones((30, 30, 3), dtype=np.uint8)

res = n / m

/tmp/ipykernel_96280/1401483766.py:4: RuntimeWarning: divide by zero encountered in divide
  res = n / m


In [ ]:
base_path = f"data/Exp2_GuyGilad_logs_yolo"
input_bboxes_path = f"{base_path}/bboxes2.csv"

In [ ]:
from evaluation.simulator import *
from evaluation.sim_controllers import *
from utils.path_utils import Files
from evaluation.config import *

experiment_config = ExperimentConfig.load_json(f"{base_path}/experiment_config.json")
# yolo_config = YoloConfig.load_json(f"{base_path}/yolo_config.json")

time_config = TimingConfig(
    imaging_time_ms=100,
    pred_time_ms=15,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.22, 0.22),
    experiment_config=experiment_config,
)

log_config = LogConfig(
    root_folder="logs",
    save_mic_view=False,
    save_cam_view=False,
    save_err_view=False,
)

In [ ]:
time_config.save_json("logs/time_config.json")

In [ ]:
from pprint import pprint

pprint(time_config)

In [ ]:
weights = np.asanyarray([0.064, 0.0, 0.0, 0.001, 0.0, 0.001, 0.698, 0.0, 0.713])
               
sample_times = np.asanyarray(
    [
        0,
        5,
        9,
        -time_config.cycle_length + 0,
        -time_config.cycle_length + 5,
        -time_config.cycle_length + 9,
        -2 * time_config.cycle_length + 0,
        -2 * time_config.cycle_length + 5,
        -2 * time_config.cycle_length + 9,
    ]
)

In [ ]:
from evaluation.sim_controllers.polyfit_controller import PolyfitController
from evaluation.motor_controllers import *
from evaluation.sim_controllers.guys_controllers import MLPController
import torch

# sim_controller = YoloController(time_config, yolo_config)
# sim_controller = SpeedController(time_config, input_bboxes_path)
sim_controller = PolyfitController(time_config, input_bboxes_path, degree=1, weights=weights, sample_times=sample_times)
# sim_controller = OptimalController(time_config, input_bboxes_path)
# sim_controller = CsvController(time_config, input_bboxes_path)
model = torch.load("data/mlpRELU")
print(model)
# sim_controller = MLPController(time_config, input_bboxes_path, model)
log_controller = LoggingController(sim_controller, log_config)

motor = SineMotorController(time_config)
sim = Simulator(
    time_config,
    experiment_config,
    log_controller,
    reader=None,
    motor_controller=motor,
)

In [ ]:
sim.run(visualize=False, wait_key=False)

In [ ]:
from evaluation.analysis import Plotter
from evaluation.config import *

pltr = Plotter(f"logs/bboxes.csv", time_config)
pltr.plot_area_vs_speed(min_speed=0.)

In [ ]:
#data = pltr.print_statistics(n=10)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_histogram(pltr, x_col:str, n:int=1, hue=None, condition=None, transform=None, **kwargs):
    data = pltr.data_prep_frames(n=n)
    
    if transform is not None:
        data = transform(data)
    if condition is not None:
            data = data[condition(data)]
    fig, ax = plt.subplots()
    sns.histplot(data=data, x=x_col, hue=hue,stat='density', **kwargs)

def plot_jointplot(pltr, x_col:str, y_col:str, n:int=1, kind:str='scatter', hue=None, condition=None, transform=None, **kwargs):
    data = pltr.data_prep_frames(n=n)
    
    if transform is not None:
        data = transform(data)
        
    if condition is not None:
            data = data[condition(data)]
    
    sns.jointplot(data=data, x=x_col, y=y_col, hue=hue, kind=kind, **kwargs)

In [ ]:
from functools import partial

def calc_speed(data: pd.DataFrame, n: int = 1) -> pd.DataFrame:
        """
        Calculate the worm speed and add it to the data.
        """
        frame_diff = data["frame"].diff(n).to_numpy()
        wrm_delta_x = data["wrm_center_x"].diff(n)
        wrm_delta_y = data["wrm_center_y"].diff(n)
        wrm_delta = np.sqrt(data["wrm_center_x"].diff(n) ** 2 + data["wrm_center_y"].diff(n) ** 2)
        data["wrm_delta"] = wrm_delta
        data["wrm_delta_x"] = wrm_delta_x
        data["wrm_delta_y"] = wrm_delta_y
        return data

plot_histogram(pltr, "wrm_delta", n=time_config.imaging_frame_num, transform=partial(calc_speed, n=time_config.imaging_frame_num))

In [ ]:
plot = pltr.plot_deviation(n=12)

In [ ]:
plot_jointplot(pltr, x_col="wrm_w", y_col="wrm_h", kind='hist')

In [ ]:
plot_histogram(pltr, "wrm_speed", 10)

In [ ]:
def to_micro_meter(data):
    data[["wrm_speed2"]] = data[["wrm_speed"]] * time_config.mm_per_px * 1000 * time_config.frames_per_sec
    return data

In [ ]:
0.87*600


In [ ]:
plot_histogram(pltr, "wrm_speed2", 10, transform=to_micro_meter)

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from evaluation.sim_controllers.polyfit_controller import WeightEvaluator

# input_offsets = np.asanyarray([0, 3, 6, 9])

input_offsets = np.asanyarray(
    [
        0,
        5,
        9,
        -time_config.cycle_length + 0,
        -time_config.cycle_length + 5,
        -time_config.cycle_length + 9,
        -2 * time_config.cycle_length + 0,
        -2 * time_config.cycle_length + 5,
        -2 * time_config.cycle_length + 9,
    ]
)


start_times = np.arange(experiment_config.num_frames // time_config.cycle_length) * time_config.cycle_length

evaluator = WeightEvaluator(
    input_bboxes_path,
    time_config,
    input_offsets=input_offsets,
    start_times=start_times,
    eval_offset= time_config.cycle_length + time_config.imaging_frame_num // 2,
    min_speed=0,
)

In [ ]:
def eval_func(weights: np.ndarray) -> float:
    return evaluator.eval(weights, deg=1)

In [ ]:
from mealpy.swarm_based.PSO import OriginalPSO
from mealpy.utils.problem import Problem
from mealpy.utils.termination import Termination
from mealpy.utils.agent import Agent
import mealpy

#optim = mealpy.ICA.OriginalICA()
optim = mealpy.PSO.OriginalPSO()

termination = Termination(max_epoch=300, max_fe=None, max_time=None, max_early_stop=100)

bounds = mealpy.FloatVar(lb=np.zeros(len(input_offsets)), ub=np.ones(len(input_offsets)))

problem = Problem(obj_func=eval_func, bounds=bounds, minimax="min")

best: Agent = optim.solve(
    problem,
    mode="swarm",
    n_workers=5,
    termination=termination,
)

In [ ]:
print(optim.g_best.target.fitness)
print((optim.g_best.solution / np.linalg.norm(best.solution)).round(3).tolist())